In [6]:
import lexnorm.models.normalise
from lexnorm.definitions import DATA_PATH
import os
import pandas as pd

train = pd.read_csv(os.path.join(DATA_PATH, "hpc/train_annotated.txt"), index_col=0)
dev = pd.read_csv(os.path.join(DATA_PATH, "hpc/dev_unannotated.txt"), index_col=0)

In [129]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
dev

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length,process,tweet,tok
an,0.377180,NaN,NaN,NaN,NaN,NaN,3.0,1.0,2,NaN,72.0,1.0,1.0,3.0,35,0,2
ca,0.255824,NaN,NaN,NaN,NaN,NaN,4.0,1.0,2,NaN,72.0,1.0,1.0,3.0,35,0,2
cab,0.261057,NaN,NaN,NaN,NaN,NaN,1.0,1.0,3,NaN,72.0,1.0,1.0,3.0,35,0,2
cam,0.299136,NaN,NaN,NaN,NaN,NaN,2.0,1.0,3,NaN,72.0,1.0,1.0,3.0,35,0,2
can,1.000000,NaN,1.0,1.0,NaN,72.0,NaN,1.0,3,1.0,72.0,1.0,1.0,3.0,35,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lulu,0.401415,NaN,NaN,NaN,NaN,NaN,14.0,1.0,4,NaN,205.0,1.0,1.0,3.0,32,9,195
pol,0.230619,NaN,NaN,NaN,NaN,NaN,9.0,1.0,3,NaN,205.0,1.0,1.0,3.0,32,9,195
smh,0.896277,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,205.0,1.0,1.0,3.0,32,9,195
sol,0.285978,NaN,NaN,NaN,NaN,NaN,3.0,1.0,3,NaN,205.0,1.0,1.0,3.0,32,9,195


In [7]:
train.fillna(0, inplace=True)
dev.fillna(0, inplace=True)
X_train = train.drop(["correct", "process", "tweet", "tok", "gold"], axis=1)
X_dev = dev.drop(["process", "tweet", "tok"], axis=1)
y_train = train["correct"]

In [132]:
from sklearn import preprocessing

feature_vals = X_train.values
scaler = preprocessing.StandardScaler().fit(feature_vals)
feature_vals = scaler.transform(feature_vals)
X_train[:] = feature_vals

feature_vals = X_dev.values
scaler = preprocessing.StandardScaler().fit(feature_vals)
feature_vals = scaler.transform(feature_vals)
X_dev[:] = feature_vals

In [5]:
svm = LinearSVC(class_weight="balanced", verbose=1).fit(X_train, y_train.values.ravel())

[LibLinear]....................................................................................................
optimization finished, #iter = 1000

Using -s 2 may be faster (also see FAQ)

Objective value = -77126.360842
nSV = 259470


/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [3]:
from joblib import dump, load
dump(svm, os.path.join(DATA_PATH, "../models/svm.joblib"))
test = load(os.path.join(DATA_PATH, "../models/svm.joblib"))
test.coef_

NameError: name 'svm' is not defined

In [134]:
preds = svm.decision_function(X_dev)

In [81]:
svm.score(X_dev, y_dev)

0.9983700596728575

In [153]:
X_train

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length
nah,2.656120,3.877197,-2.593884,-0.094405,-0.046030,-0.048241,-0.268871,-4.348835,-1.763720,-3.155060,-1.057043,0.345895,0.0,1.361154
oh,2.745444,2.535195,-2.593884,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-2.095137,-3.155060,-1.057043,0.345895,0.0,1.361154
uh,2.624856,5.219199,-2.593884,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-2.095137,-3.155060,-1.057043,0.345895,0.0,1.361154
ye ah,-1.138580,-0.148809,-2.593884,-0.094405,21.724887,-0.048241,1.360023,-4.348835,-1.100884,0.316951,-1.057043,0.345895,0.0,1.361154
ye-ah,-1.138580,-0.148809,-2.593884,-0.094405,-0.046030,-0.048241,1.767247,-4.348835,-1.100884,0.316951,-1.057043,0.345895,0.0,1.361154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
upward,0.569510,-0.148809,0.385522,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-0.769467,0.316951,-0.688526,0.345895,0.0,-0.462068
upwardly,0.702616,-0.148809,0.385522,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-0.106631,0.316951,-0.688526,0.345895,0.0,-0.462068
upwards,0.833669,-0.148809,0.385522,-0.094405,-0.046030,-0.048241,-0.268871,0.229947,-0.438049,0.316951,-0.688526,0.345895,0.0,-0.462068
us,1.122110,-0.148809,-2.593884,-0.094405,-0.046030,-0.048241,2.988917,0.229947,-2.095137,-3.155060,-0.688526,0.345895,0.0,-0.462068


In [149]:
dev["preds"] = preds
dev.loc[(dev.process == 0) & (dev.tweet == 0) & (dev.tok == 4)]

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length,process,tweet,tok,preds
citch,0.756863,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0.0,1.0,1.0,1.0,2.0,0,0,4,-1.731161
co,0.240537,0.0,0.0,0.0,0.0,0.0,14.0,0.0,2,0.0,1.0,1.0,1.0,2.0,0,0,4,-2.434154
hoe,0.741933,4.0,0.0,0.0,0.0,0.0,0.0,1.0,3,0.0,1.0,1.0,1.0,2.0,0,0,4,-1.538175
lo,0.361795,0.0,0.0,0.0,0.0,0.0,13.0,1.0,2,0.0,1.0,1.0,1.0,2.0,0,0,4,-1.583027
nigga,0.742659,3.0,0.0,0.0,0.0,0.0,0.0,1.0,5,0.0,1.0,1.0,1.0,2.0,0,0,4,-1.389208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yowl's,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,6,1.0,1.0,1.0,1.0,2.0,0,0,4,-1.555294
yowled,0.283298,0.0,1.0,0.0,0.0,0.0,0.0,1.0,6,1.0,1.0,1.0,1.0,2.0,0,0,4,-1.327322
yowling,0.225846,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7,1.0,1.0,1.0,1.0,2.0,0,0,4,-1.390141
yowls,0.204925,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5,1.0,1.0,1.0,1.0,2.0,0,0,4,-1.373803


In [137]:
# get top prediction for each raw token
predictions = dev.sort_values("preds", ascending=False).drop_duplicates(["process", "tweet", "tok"])
pred_tokens = predictions.sort_values(["process", "tweet", "tok"]).index.tolist()
pred_tokens

['brother',
 'get',
 'out',
 'yo',
 'feelings',
 'lol',
 'manan',
 'dund',
 'xaragdax',
 'ter',
 'uuliin',
 'oroid',
 'minii',
 'aav',
 'xezee',
 'neg',
 'cagt',
 'zogsoj',
 'baisan',
 'photo',
 'by',
 'why',
 'dese',
 'niggers',
 'think',
 'dey',
 'doin',
 'summn',
 "it's",
 'about',
 'more',
 'than',
 'number',
 'brother',
 'and',
 "i'm",
 'not',
 'talkn',
 'about',
 'statistics',
 "i'm",
 'talkn',
 'about',
 'skill',
 'level',
 'yes',
 'omg',
 'i',
 "don't",
 'want',
 'him',
 'feeling',
 'unappreciated',
 'or',
 'stuff',
 'like',
 'that',
 'lmao',
 'the',
 'whole',
 'time',
 'actually',
 'we',
 'need',
 'to',
 'start',
 'our',
 'own',
 'team',
 "y'all",
 'were',
 'wilding',
 'last',
 'nigjt',
 'yoh',
 'niya',
 'ja',
 'saying',
 'kau',
 'aku',
 'with',
 'couple',
 'is',
 'actually',
 'sweet',
 'k',
 'michelle',
 'met',
 'yo',
 'match',
 'that',
 'fonk',
 'rana',
 'samaha',
 'ignore',
 'all',
 'the',
 'criticism',
 'a',
 'spotlight',
 'can',
 'never',
 'see',
 'the',
 'shadows',
 'any

In [22]:
from lexnorm.data.normEval import loadNormData

raw, norm = loadNormData(os.path.join(DATA_PATH, "raw/dev.norm"))

In [23]:
from lexnorm.models.filtering import is_eligible

pred_tokens_iter = iter(pred_tokens)

pred_tweets = []

for tweet in raw:
    pred_tweet = []
    for tok in tweet:
        if is_eligible(tok):
            pred_tweet.append(next(pred_tokens_iter))
        else:
            pred_tweet.append(tok)
    pred_tweets.append(pred_tweet)

In [24]:
from lexnorm.data.normEval import evaluate

evaluate(raw, norm, pred_tweets)
# pretty bad. Some reasons: not good enough feature extraction, using a linear svm when data might not be linearly seperable in our feature space (may need different kernel), set non-calculated/absent values to zero which can be picked up by a random forest but NOT by most models! May need to give highest possible value instead.

Baseline acc.(LAI): 93.10
Accuracy:           96.28
ERR:                46.13


(0.9309630275929763, 0.9628094666812084, 0.4612954186413895)

In [157]:
from sklearn.inspection import DecisionBoundaryDisplay
DecisionBoundaryDisplay.from_estimator(svm, X_dev[:, :2])

InvalidIndexError: (slice(None, None, None), slice(None, 2, None))

In [4]:
from joblib import load
import os
from lexnorm.definitions import DATA_PATH
rd_clf = load(os.path.join(DATA_PATH, "../models/rf.joblib"))

In [198]:
rd_clf.estimators_

[DecisionTreeClassifier(max_features='sqrt', random_state=1608637542),
 DecisionTreeClassifier(max_features='sqrt', random_state=1273642419),
 DecisionTreeClassifier(max_features='sqrt', random_state=1935803228),
 DecisionTreeClassifier(max_features='sqrt', random_state=787846414),
 DecisionTreeClassifier(max_features='sqrt', random_state=996406378),
 DecisionTreeClassifier(max_features='sqrt', random_state=1201263687),
 DecisionTreeClassifier(max_features='sqrt', random_state=423734972),
 DecisionTreeClassifier(max_features='sqrt', random_state=415968276),
 DecisionTreeClassifier(max_features='sqrt', random_state=670094950),
 DecisionTreeClassifier(max_features='sqrt', random_state=1914837113),
 DecisionTreeClassifier(max_features='sqrt', random_state=669991378),
 DecisionTreeClassifier(max_features='sqrt', random_state=429389014),
 DecisionTreeClassifier(max_features='sqrt', random_state=249467210),
 DecisionTreeClassifier(max_features='sqrt', random_state=1972458954),
 DecisionTreeC

In [12]:
rd_clf.oob_score_

0.9995261407341941

In [200]:
rd_clf.decision_path(X_dev)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.5s finished


KeyboardInterrupt: 

In [223]:
rd_clf.feature_names_in_

array(['cosine_to_orig', 'embeddings_rank', 'from_clipping',
       'from_original_token', 'from_split', 'norms_seen',
       'spellcheck_rank', 'in_lexicon', 'length', 'same_order',
       'orig_norms_seen', 'orig_in_lexicon', 'orig_same_order',
       'orig_length'], dtype=object)

In [17]:
rd_clf.feature_importances_

array([2.52070851e-01, 4.31976125e-03, 7.92054467e-03, 1.64534043e-01,
       1.61258200e-04, 4.01924066e-01, 3.03852361e-03, 7.35963146e-03,
       6.02726879e-02, 9.02983575e-03, 3.24685801e-02, 7.44901879e-03,
       0.00000000e+00, 4.94511981e-02])

In [8]:
preds = rd_clf.predict_proba(X_dev)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.8s finished


In [9]:
dev["preds"] = preds[:, 1]
predictions = dev.sort_values("preds", ascending=False).drop_duplicates(["process", "tweet", "tok"])
pred_tokens = predictions.sort_values(["process", "tweet", "tok"]).index.tolist()
pred_tokens

['brother',
 'get',
 'out',
 'you',
 'feelings',
 'lol',
 'manan',
 'dund',
 'paradox',
 'ter',
 'insulin',
 'oroid',
 'minii',
 'aav',
 'xezee',
 'neg',
 'cagt',
 'couscous',
 'baisan',
 'photo',
 'by',
 'why',
 'dese',
 'niggers',
 'think',
 'dey',
 'doing',
 'summn',
 "it's",
 'about',
 'more',
 'than',
 'number',
 'brother',
 'and',
 "i'm",
 'not',
 'talking',
 'about',
 'statistics',
 "i'm",
 'talking',
 'about',
 'skill',
 'level',
 'yes',
 'omg',
 'i',
 "don't",
 'want',
 'him',
 'feeling',
 'unappreciated',
 'or',
 'stuff',
 'like',
 'that',
 'lmao',
 'the',
 'whole',
 'time',
 'actually',
 'we',
 'need',
 'to',
 'start',
 'our',
 'own',
 'team',
 "y'all",
 'were',
 'wilding',
 'last',
 'nigjt',
 'yoh',
 'niya',
 'ja',
 'saying',
 'kau',
 'aku',
 'with',
 'couple',
 'is',
 'actually',
 'sweet',
 'k',
 'michelle',
 'met',
 'you',
 'match',
 'that',
 'fonk',
 'rana',
 'samaha',
 'ignore',
 'all',
 'the',
 'criticism',
 'a',
 'spotlight',
 'can',
 'never',
 'see',
 'the',
 'shadow

In [35]:
from lexnorm.generate_extract.filtering import is_eligible
from lexnorm.data.normEval import loadNormData, evaluate
raw, norm = loadNormData(os.path.join(DATA_PATH, "raw/dev.norm"))
# pred_tokens_iter = iter(pred_tokens)
#
# pred_tweets = []
#
# for tweet in raw:
#     pred_tweet = []
#     for tok in tweet:
#         if is_eligible(tok):
#             pred_tweet.append(next(pred_tokens_iter))
#         else:
#             pred_tweet.append(tok)
#     pred_tweets.append(pred_tweet)

In [14]:
# better!!
evaluate(raw, norm, pred_tweets)

Baseline acc.(LAI): 93.10
Accuracy:           96.28
ERR:                46.13


(0.9309630275929763, 0.9628094666812084, 0.4612954186413895)

In [234]:
preds

array([[1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ],
       ...,
       [0.98, 0.02],
       [1.  , 0.  ],
       [1.  , 0.  ]])

In [39]:
import pandas as pd
import os
from lexnorm.definitions import DATA_PATH

train = pd.read_csv(os.path.join(DATA_PATH, "hpc/train_processed_annotated.txt"), index_col=0)
dev = pd.read_csv(os.path.join(DATA_PATH, "hpc/dev_processed.txt"), index_col=0)

In [29]:
train.loc[(train['process'] == 63) & (train['tweet'] == 0) & (train['tok'] == 2)]

AttributeError: 'function' object has no attribute 'loc'

In [25]:
from lexnorm.data.word2vec import get_vectors

vectors = get_vectors(os.path.join(DATA_PATH, "raw/train.norm"))

In [26]:
from lexnorm.generate_extract.modules import word_embeddings

word_embeddings("yeah", vectors)

/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


,cosine_to_orig,embeddings_rank
yea,0.912848,1
yes,0.821176,2
oh,0.817392,3
nah,0.798594,4
uh,0.792015,5


In [40]:
# from lexnorm.models.random_forest import train

dev.groupby(["process", "tweet", "tok"]).size()

process  tweet  tok
0        0      1       16
                2       34
                3      291
                4      113
                5       11
                      ... 
58       9      10      49
                11      55
                12      38
                13       6
                14       1
Length: 6876, dtype: int64

In [42]:
count = 0

for tweet in raw:
    for tok in tweet:
        if is_eligible(tok):
            count += 1

print(count)
# as expected - candidates generated for every eligible raw token

6876


In [62]:
from lexnorm.models.normalise import prep_test, normalise
import joblib

from lexnorm.models import random_forest
clf = joblib.load(os.path.join(DATA_PATH, "../models/rf.joblib"))
dev_X = prep_test(os.path.join(DATA_PATH, "hpc/dev_processed.txt"))
dev = pd.read_csv(os.path.join(DATA_PATH, "hpc/dev_processed.txt"), index_col=0)
preds = random_forest.predict(clf, dev_X, dev)
predictions = normalise(raw, preds)

NameError: name 'lexnorm' is not defined

In [59]:
evaluate(raw, norm, predictions, verbose=True)

yo your you
xaragdax xaragdax paradox
uuliin uuliin insulin
zogsoj zogsoj couscous
dese these dese
dey they dey
summn something summn
nigjt night nigjt
shettima shettima estimate
realised realized realised
hella hell of a lot of hella
bestie best friend bestie
chillen chilling chillen
niggra nigger niggra
yellowcard yellow card yellowcard
da da the
factor factor factoring
darlin darling darlin
smyle smile smyle
sadest saddest sadest
tenerezza tenerezza containerize
wkeend weekend keen
bro brother bro
babyface baby face babyface
orrr or orrr
noo no noo
readers readers reviewers
ammmazing amazing ammmazing
actully actually actully
id i'd id
ohhh ohhh oh
contactmusic contactmusic contact music
datpiff the piff datpiff
soundcloud sound cloud soundcloud
sunshelter sunshelter sun shelter
jumpseat jumpseat jump seat
sensual sensual seductive
toribash toribash retribution
diy do it yourself diy
oookay okay oookay
headass head ass headass
niyata niyata lanyard
flavour flavor flavour
n n and
bro

(0.9309630275929763, 0.9629185298287709, 0.4628751974723546)

In [66]:
dev = pd.read_csv(os.path.join(DATA_PATH, "hpc/train_processed_annotated.txt"), index_col=0)
print(dev.groupby(["process", "tweet", "tok"]).size().mean())
dev = pd.read_csv(os.path.join(DATA_PATH, "hpc/train_annotated.txt"), index_col=0)
print(dev.groupby(["process", "tweet", "tok"]).size().mean())

114.09050270864114
113.20335644201992


In [83]:
dev = pd.read_csv(os.path.join(DATA_PATH, "hpc/train_annotated.txt"), index_col=0)
dev.loc[(dev['process'] == 63) & (dev['tweet'] == 0) & (dev['tok'] == 2)]

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length,correct,process,tweet,tok,gold
nah,0.798594,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,18.0,1.0,1.0,4.0,NaN,63,0,2,yeah
oh,0.817392,2.0,NaN,NaN,NaN,NaN,NaN,1.0,2,NaN,18.0,1.0,1.0,4.0,NaN,63,0,2,yeah
uh,0.792014,4.0,NaN,NaN,NaN,NaN,NaN,1.0,2,NaN,18.0,1.0,1.0,4.0,NaN,63,0,2,yeah
ye ah,NaN,NaN,NaN,NaN,1.0,NaN,4.0,NaN,5,1.0,18.0,1.0,1.0,4.0,NaN,63,0,2,yeah
ye-ah,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5,1.0,18.0,1.0,1.0,4.0,NaN,63,0,2,yeah
yea,0.912848,0.0,NaN,NaN,NaN,NaN,0.0,1.0,3,NaN,18.0,1.0,1.0,4.0,NaN,63,0,2,yeah
yea h,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,5,1.0,18.0,1.0,1.0,4.0,NaN,63,0,2,yeah
yea-h,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,5,1.0,18.0,1.0,1.0,4.0,NaN,63,0,2,yeah
yeah,1.000000,NaN,1.0,1.0,NaN,18.0,NaN,1.0,4,1.0,18.0,1.0,1.0,4.0,1.0,63,0,2,yeah
yeah's,0.587786,NaN,1.0,NaN,NaN,NaN,NaN,1.0,6,1.0,18.0,1.0,1.0,4.0,NaN,63,0,2,yeah


In [82]:
dev = pd.read_csv(os.path.join(DATA_PATH, "hpc/train_processed_annotated.txt"), index_col=0)
dev.loc[(dev['process'] == 63) & (dev['tweet'] == 0) & (dev['tok'] == 2)]

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length,raw,gold,correct,process,tweet,tok
leah,0.384448,NaN,NaN,NaN,NaN,NaN,5.0,1.0,4,NaN,18.0,1.0,1.0,4.0,yeah,yeah,NaN,63,0,2
nah,0.798594,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,18.0,1.0,1.0,4.0,yeah,yeah,NaN,63,0,2
oh,0.817392,4.0,NaN,NaN,NaN,NaN,NaN,1.0,2,NaN,18.0,1.0,1.0,4.0,yeah,yeah,NaN,63,0,2
uh,0.792014,6.0,NaN,NaN,NaN,NaN,NaN,1.0,2,NaN,18.0,1.0,1.0,4.0,yeah,yeah,NaN,63,0,2
ye ah,NaN,NaN,NaN,NaN,1.0,NaN,6.0,NaN,5,1.0,18.0,1.0,1.0,4.0,yeah,yeah,NaN,63,0,2
ye-ah,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,5,1.0,18.0,1.0,1.0,4.0,yeah,yeah,NaN,63,0,2
yea,0.912848,1.0,NaN,NaN,NaN,NaN,1.0,1.0,3,NaN,18.0,1.0,1.0,4.0,yeah,yeah,NaN,63,0,2
yea h,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,5,1.0,18.0,1.0,1.0,4.0,yeah,yeah,NaN,63,0,2
yea-h,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,5,1.0,18.0,1.0,1.0,4.0,yeah,yeah,NaN,63,0,2
yeah,1.000000,2.0,NaN,1.0,NaN,18.0,NaN,1.0,4,1.0,18.0,1.0,1.0,4.0,yeah,yeah,1.0,63,0,2


In [77]:
clf.predict_log_proba("test")

/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: could not convert string to float: 'test'

In [116]:
import importlib
from lexnorm.generate_extract import modules
from lexnorm.generate_extract import candidate_generation
importlib.reload(modules)

modules.word_embeddings("the", vectors)

/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


,cosine_to_orig,embeddings_rank
The,0.757326,1
